# Validation of t-SNE overlap approach
Make t-SNE of all freesolv quintuplicates. Make a split such that the test quints are not well-represented in the training set. Check if these are poorly predicted on by a model. Then incrementally add back the most un-represented cases and update statistics. 

In [1]:
import glob
import csv
import re
import pandas as pd
import itertools
import numpy as np
from tqdm.notebook import tqdm
import csv
import matplotlib.pyplot as plt 
import seaborn as sns 
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}

import sklearn
from sklearn.manifold import TSNE
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from scipy import stats

import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from tensorflow import keras
from tensorflow.keras import layers

from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, rdFMCS, AllChem, rdmolfiles, Descriptors, rdchem

ImportError: Traceback (most recent call last):
  File "/home/jscheen/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/jscheen/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/jscheen/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/ins

In [ ]:
quints_fps = pd.read_csv("output/quints_fps.csv", header=None)
quints_infos = pd.read_csv("output/quints_infos.csv", names=["set", "pertname", "pertsmarts", "num_ha", "sem"])

quints_whole_df = pd.concat([quints_infos, quints_fps], axis=1)

# drop NaN columns (happens with molprop generation where (error) strings can't be subtracted)
quints_whole_df = quints_whole_df.dropna(axis=1)

# drop rows where SEM == 0.0. It seems some very large perturbations get this value too, so makes training noisy.
quints_whole_df = quints_whole_df[quints_whole_df["sem"] > 0.0001]

# drop columns where all values are 0.
quints_whole_df = quints_whole_df.loc[:, (quints_whole_df != 0).any(axis=0)]

# TMP DROP DUPLICATES --> should be fewer duplicates when we move up to more features.
quints_whole_df = quints_whole_df.drop_duplicates(subset=quints_whole_df.columns.difference(['sem','set','pertname','pertsmarts']))

quints_whole_df

### Discretize SEM label into categorical bins

In [ ]:
### TMP DISCRETIZE BY STRATIFICATION:
n_bins=20

binned_sem = pd.qcut(quints_whole_df["sem"], n_bins, labels=False)
quints_whole_df["sem_bin"] = binned_sem
print("Bin, Min, Max, Volume")
for n_bin, df_group in quints_whole_df.groupby(by="sem_bin"):
    print(n_bin, round(min(df_group["sem"].values), 2), round(max(df_group["sem"].values), 2), len(df_group))
    

In [ ]:
### TMP SLIM DOWN DOMINANT CLASSES
# quints_whole_df.sort_values(by="sem_bin")
# quints_whole_df = quints_whole_df.groupby("sem_bin").head(100)



In [ ]:
quints_fps = quints_whole_df.drop(["set", "pertname", "pertsmarts", "num_ha", "sem", "sem_bin"], axis=1)
quints_fps = quints_fps.values
quints_infos = quints_whole_df[["set", "pertname", "pertsmarts", "num_ha", "sem", "sem_bin"]]

In [ ]:
def fitTSNE(fps):

    # https://iwatobipen.wordpress.com/2019/12/22/clustering-molecules-with-hdbscan-and-predict-chemical-space-with-scikit-learn-chemoinformatics-rdkit/
    tsne = TSNE(random_state=42)

    res = tsne.fit_transform(fps)
    
    return res

In [ ]:
res = fitTSNE(quints_fps)

In [ ]:
upper_test_indices = []
lower_test_indices = []
train_indices = []

colours = []

for i, (x, y) in enumerate(zip(res[:,0], res[:,1])):
    if 30 < x < 40 and -10 < y < 5:
        upper_test_indices.append(i)
        colours.append("Test set")
    elif -10 < x < -2 and 25 < y < 35:
        lower_test_indices.append(i)
        colours.append("Test set")
    else:
        train_indices.append(i)
        colours.append("Training set")

plt.figure(figsize=(8, 8))
sns.scatterplot(x=res[:,0], y=res[:,1], hue=colours, **plot_kwds)

plt.xlabel("t-SNE 0")
plt.ylabel("t-SNE 1")
plt.title("tSNE plot of simulated FreeSolv quintuplicates")
plt.show()

### Split into train (orange) and test (blue)

In [ ]:
def takeSubset(indices, quints_whole_df):
    """Take a selection of a dataframe using indices"""
    subset = quints_whole_df.iloc[indices]
    
    return subset
    

In [ ]:
def takeInfo(perts_df):
    """from the input dataframe, return arrays of fingerprints and SEMs"""
    sems = perts_df["sem_bin"].values
    
    # fps is a bit more involved. Remove everything but the FP columns, return as 2d array.
    fps_df = perts_df.drop(["set", "pertname", "pertsmarts", "num_ha", "sem", "sem_bin"], axis=1)
    fps = fps_df.values

    return fps, sems

In [ ]:
def preProcessSets(test1, test2, train):
    """standardises and reduces dimensionality to 95% VE; returns arrays"""
    
    #### fit the scaler on training set.
    scaler = preprocessing.StandardScaler()
    train_scaled = scaler.fit_transform(train)
    
    # transform both the training set and the test sets.
    test1_scaled = scaler.transform(test1)
    test2_scaled = scaler.transform(test2)
    
    #### fit PCA on training set with 95% variance explained.
    pca = PCA(n_components=0.95)
    train_preprocessed = pca.fit_transform(train_scaled)
    
    # transform both the training set and the test sets.
    test1_preprocessed = pca.transform(test1_scaled)
    test2_preprocessed = pca.transform(test2_scaled)
    
    return test1_preprocessed, test2_preprocessed, train_preprocessed

In [ ]:
# get fps, sem, preprocess (i.e. scale and reduce dimensionality)
# upper_test_fps, upper_test_sems = takeInfo(takeSubset(upper_test_indices, quints_whole_df))
# lower_test_fps, lower_test_sems = takeInfo(takeSubset(lower_test_indices, quints_whole_df))
# train_fps, train_sems = takeInfo(takeSubset(train_indices, quints_whole_df))

# test_set1, test_set2, train_set = preProcessSets(upper_test_fps, lower_test_fps, train_fps)

# n_classes = len(set(train_sems))


In [ ]:
## TMP RANDOM SPLIT INSTEAD OF TSNE SPLIT
from sklearn.model_selection import train_test_split
whole_set, whole_set_sem_bins = takeInfo(takeSubset(np.array(range(len(quints_whole_df))), quints_whole_df))

train_set, test_set1, train_sems, upper_test_sems = train_test_split(whole_set, whole_set_sem_bins, test_size=0.3, random_state=42)
n_classes = len(set(train_sems))

In [ ]:
def build_model(loss='sparse_categorical_crossentropy', 
                lr=1e-3, 
                num_neurons_1=10,
                num_neurons_2=None,
                num_neurons_3=None,
                num_neurons_4=None,
                input_shape=train_set.shape[1],
                act="relu",
                optimizer="adam"):
    # standard setup FF DNN with input shape of input data.
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape))
    
    model.add(keras.layers.Dropout(0.4))
    # first layer.
    model.add(keras.layers.Dense(num_neurons_1, activation=act))
    model.add(keras.layers.Dropout(0.4))
    
    # second layer.
    if num_neurons_2:
        model.add(keras.layers.Dense(num_neurons_2, activation=act))
        model.add(keras.layers.Dropout(0.4))
    
    # third layer.
    if num_neurons_3:
        model.add(keras.layers.Dense(num_neurons_3, activation=act))
        model.add(keras.layers.Dropout(0.4))
        
    # fourth layer.
    if num_neurons_4:
        model.add(keras.layers.Dense(num_neurons_4, activation=act))
        model.add(keras.layers.Dropout(0.4))
    
    # standard last linear layer for regression.
    #model.add(keras.layers.Dense(1, activation="linear"))
    
    # standard last layer for classification.
    model.add(keras.layers.Dense(n_classes, activation="softmax"))
    

    model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    return model

keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model, epochs=500, verbose=0)

params_distrib = {
    "loss": ['sparse_categorical_crossentropy', 'poisson'],
    "lr": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
    "num_neurons_1": [5, 10, 20, 40],
    "num_neurons_2": [None, 8, 20, 40, 80],
    "num_neurons_3": [None, 8, 20, 40, 80],
    "num_neurons_4": [None, 8, 20, 40, 80],
    "act": ["relu", "tanh", "sigmoid"],
    "optimizer": ["adam", "SGD"]
}
rnd_search_cv = RandomizedSearchCV(keras_reg, params_distrib, n_iter=20, n_jobs=5, cv=5, verbose=2)
rnd_result = rnd_search_cv.fit(np.array(train_set), np.array(train_sems), 
                               ) 
print(rnd_result.best_params_)
print(rnd_result.best_score_)

In [ ]:
# see ALL validation scores.
rnd_result.cv_results_

## Use the indicated hyperparameters to overfit a single DNN, check loss.

In [ ]:
def build_model_opt(loss, 
                lr, 
                num_neurons_1,
                num_neurons_2,
                num_neurons_3,
                num_neurons_4,
                act,
                optimizer,
                input_shape=train_set.shape[1]):
    # standard setup FF DNN with input shape of input data.
    model = keras.models.Sequential()
    model.add(keras.layers.Dropout(0.4, input_shape=(input_shape,)))
    
    # first layer.
    model.add(keras.layers.Dense(num_neurons_1, activation=act))
    model.add(keras.layers.Dropout(0.4))
    
    # second layer.
    if num_neurons_2:
        model.add(keras.layers.Dense(num_neurons_2, activation=act))
        model.add(keras.layers.Dropout(0.4))
    
    # third layer.
    if num_neurons_3:
        model.add(keras.layers.Dense(num_neurons_3, activation=act))
        model.add(keras.layers.Dropout(0.4))
        
    # fourth layer.
    if num_neurons_4:
        model.add(keras.layers.Dense(num_neurons_4, activation=act))
        model.add(keras.layers.Dropout(0.4))
    
    # standard last linear layer for regression.
    #model.add(keras.layers.Dense(1, activation="linear"))
    
    # standard last layer for classification.
    model.add(keras.layers.Dense(n_classes, activation="softmax"))
    optimizer = keras.optimizers.Adam(lr=0.0001)

    model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    return model

In [ ]:
%%time
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50)
dnn_model = build_model_opt(**rnd_result.best_params_)
history = dnn_model.fit(
    np.array(train_set), np.array(train_sems),
    validation_split=0.2,
    verbose=0, batch_size=64, epochs=5000,
    #callbacks=[callback]
    )


In [ ]:
def plot_loss(history, lim):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    if lim:
        plt.xlim(lim)
    #plt.ylim(0,1)
    plt.xlabel('Epoch')
    plt.ylabel('Cost')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_loss(history, [0, 5000])

#plot_train_scatter(dnn_model, train_set, train_sems)

In [ ]:
for pred in dnn_model.predict(test_set1):
    print(np.argmax(pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.svm import SVC 

model = SVC()


param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 1) 
  
# fitting the model for grid search 
grid.fit(np.array(train_set), np.array(train_sems)) 

In [ ]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

In [ ]:
grid_predictions = grid.predict(test_set1) 
  
# print classification report 
print(classification_report(upper_test_sems, grid_predictions)) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state = 1)

n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30, 50]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 5, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(np.array(train_set), np.array(train_sems))

In [ ]:
# print best parameter after tuning 
print(gridF.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(gridF.best_estimator_) 

In [ ]:
grid_predictions = gridF.predict(test_set1) 
  
# print classification report 
print(classification_report(upper_test_sems, grid_predictions)) 

In [ ]:
grid_predictions